In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genmc-logit-training/test.jsonl
/kaggle/input/genmc-logit-training/pytorch_model-001.bin
/kaggle/input/genmc-logit-training/dev (2).jsonl
/kaggle/input/genmc-logit-training/train (1).jsonl
/kaggle/input/genmc-logit-training/GenMC/GenMC/run_genmc.py
/kaggle/input/genmc-logit-training/GenMC/GenMC/utils.py
/kaggle/input/genmc-logit-training/GenMC/GenMC/requirements.txt
/kaggle/input/genmc-logit-training/GenMC/GenMC/model/modeling_genmc.py
/kaggle/input/genmc-logit-training/GenMC/GenMC/model/modeling_transformer.py
/kaggle/input/genmc-logit-training/GenMC/GenMC/data/obqa/dev.jsonl
/kaggle/input/genmc-logit-training/GenMC/GenMC/data/obqa/test.jsonl
/kaggle/input/genmc-logit-training/GenMC/GenMC/data/obqa/train.jsonl
/kaggle/input/genmc-teacher-logits-and-clues/dev.jsonl
/kaggle/input/genmc-teacher-logits-and-clues/test.jsonl
/kaggle/input/genmc-teacher-logits-and-clues/train.jsonl


In [2]:
cd "/kaggle/input/genmc-logit-training/GenMC/GenMC"


/kaggle/input/genmc-logit-training/GenMC/GenMC


In [3]:
ls

data/  model/  requirements.txt  run_genmc.py  utils.py


In [4]:
!pip install rouge

In [5]:
from transformers import T5Tokenizer
from tqdm import trange
import os
import random
import torch
from utils import read_dataset, save_dataset, read_dataset, set_seed, save_model
from model.modeling_genmc import GenMC
from run_genmc import get_input_feature
import json
import argparse
from tqdm.notebook import tqdm

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [6]:
input_path = "/kaggle/input/genmc-teacher-logits-and-clues/"

In [7]:
from torch.utils.data import Dataset


class OBQADataset(Dataset):
    def __init__(self, dataset, has_H_QC = False):
        self.dataset = dataset
    def __len__(self):
        return len(dataset)

    def __getitem__(self, idx):
        features = {} 
        features["id"] = dataset[idx]["id"]
        features["question"] = dataset[idx]["question"]
        features["answerKey"] = dataset[idx]["answerKey"]
        
        features["logits"] = torch.Tensor(dataset[idx]["logits"])
        features["H_QC"] = torch.Tensor(dataset[idx]["H_QC"])
    
        return features

In [8]:
train_dataset = read_dataset(input_path+"train.jsonl")
print(len(train_dataset))
val_dataset = read_dataset(input_path+"dev.jsonl")
print(len(val_dataset))
test_dataset = read_dataset(input_path+"test.jsonl")
print(len(test_dataset))

2478
500
500


In [9]:
# from torch.utils.data import DataLoader
# train_batch_size = 8
# eval_batch_size = 8

# train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=eval_batch_size, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=eval_batch_size, shuffle=True)

In [10]:
from transformers import T5ForConditionalGeneration
model_path = "google-t5/t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:

def get_input_feature(samples, max_source_length, max_len_gen, choice_num, external_sent_num=None):
    sep = ' \\n '
    output_clue = []
    answers = []
    input_ids_q, attention_mask_q = [], []
    input_ids_qo, attention_mask_qo = [], []
    logits, H_QC, H_QC_mask = [], [], []
    for sample in samples:
        if 'answerKey' in sample:
            answerKey = sample['answerKey']
        else:
            answerKey = "A"
        question = sample['question']['stem']
        while len(sample['question']['choices']) < choice_num:
            sample['question']['choices'].append({"text": "error", "para": "", "label": chr(ord('A')+len(sample)-1)})
        for o_i, (opt, opt_name) in enumerate(zip(sample['question']['choices'], 'ABCDEFGH'[:choice_num])):
            option = opt['text']
            content = ""
            if external_sent_num is not None and 'para' in opt:
                para = opt["para"]
                if isinstance(para, list):
                    if len(para) > external_sent_num:
                        para = para[:external_sent_num]
                    content = sep + " ".join(para)
                elif isinstance(para, str):
                    para = para.split(".")
                    if len(para) > external_sent_num:
                        para = para[:external_sent_num]
                    content = sep + " ".join(para)
                else:
                    print('lack retrieval')
                    # exit(0)
            input_ids_qo.append(question + sep + option + content)


        input_ids_q.append(question + sep)
        if answerKey in '123456':
            answer = ord(answerKey) - ord('1')
        else:
            answer = ord(answerKey) - ord('A')
        answers.append(answer)
        output_clue.append(sample['question']['choices'][answer]['text'])
        logits.append(sample["logits"])
        H_QC.append(sample["H_QC"])
        H_QC_mask.append(sample["output_sequences_mask"])

    def tokenizer_fun(input_ids, max_len):
        encoding = tokenizer(input_ids,
                             padding='longest',
                             max_length=max_len,
                             truncation=True,
                             return_tensors="pt")
        ids = encoding.input_ids.to(device)
        mask = encoding.attention_mask.to(device)
        return ids, mask

    q_ids, q_mask = tokenizer_fun(input_ids_q, max_source_length)
    qo_ids, qo_mask = tokenizer_fun(input_ids_qo, max_source_length)
    clue_ids, _ = tokenizer_fun(output_clue, max_len_gen)
    clue_ids = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in
        clue_ids
    ]
    clue_ids = torch.tensor(clue_ids, dtype=torch.long).to(device)
    answers = torch.tensor(answers, dtype=torch.long).to(device)
    H_QC = torch.tensor(H_QC)
    H_QC = H_QC.reshape((qo_ids.shape[0],H_QC.shape[2], H_QC.shape[3])).to(device)
    H_QC_mask = torch.tensor(H_QC_mask).reshape(qo_ids.shape[0],-1).to(device)
#     print("Hqc",H_QC.shape)
#     print("Hqc mask",H_QC_mask.shape)
    logits = torch.tensor(logits).to(device)
    return q_ids, q_mask, qo_ids, qo_mask, clue_ids, answers, output_clue, logits, H_QC, H_QC_mask

In [12]:
from model.modeling_transformer import SemanticMatch
from torch import nn
from torch.nn import functional as F
def kl_ce_loss(t_logits, s_logits, y, alpha, beta, t=1.0):
    """
    Computes: alpha * KL-Divergence + beta * Cross-Entropy

    Args:
        t_logits (Torch.tensor): Teacher logits
        s_logits (Torch.tensor): Student logits
        y (Torch.tensor): Ground-truth labels
        alpha (float): KL-divergence loss weight
        beta (float): Cross-entropy loss weight
        t (float): Temperature
    """
    # KL Divergence
    t_probs = F.softmax(t_logits / t, dim=1)
    s_probs = F.log_softmax(s_logits / t, dim=1)
    kl_loss = F.kl_div(s_probs, t_probs, reduction="batchmean") * (t**2)

    # Cross-Entropy
    ce = F.cross_entropy(s_logits, y, reduction="mean")

    return alpha * kl_loss + beta * ce

class StudentMC(nn.Module):
    def __init__(self, num_hidden_layers, alpha, beta):
        super(StudentMC, self).__init__()
        self.alpha = alpha
        self.beta = beta
        
        device = torch.device("cuda:0")
        
        self.t5_model = T5ForConditionalGeneration.from_pretrained(model_path)
        dim = self.t5_model.config.d_model
        self.dim_reduction = nn.Linear(2*dim, dim).to(device)
        self.dim_reduction.device = device
        self.semantic_matching = SemanticMatch(dim, num_hidden_layers).to(device)
        self.semantic_matching.device = device
        self.option_linear = nn.Linear(dim, 1).to(device)
        self.choice_num = 5
        
        n_gpu = torch.cuda.device_count()
        layer_num = self.t5_model.config.num_layers
        layer_per_gpu = layer_num // n_gpu
        layer_per_gpu_remainder = layer_num % n_gpu
        device_map = {}
        cur_layer = 0
        for n in range(n_gpu):
            device_map[n] = []
            if n < layer_per_gpu_remainder:
                layer_assigned = layer_per_gpu+1
            else:
                layer_assigned = layer_per_gpu

            for i in range(layer_assigned):
                device_map[n].append(cur_layer)
                cur_layer += 1
        self.t5_model.parallelize(device_map)
        
    def forward(self, q_ids, q_mask, qo_ids, qo_mask, logits, H_QC, H_QC_mask,  answers=None):
        local_device = self.t5_model.encoder.device
#         print(qo_ids.shape)
        t5_output = self.t5_model.encoder(input_ids=qo_ids.to(local_device), attention_mask=qo_mask.to(local_device))
        encoder_qo = t5_output[0]
        
        decoder_qo = self.dim_reduction(H_QC.to(local_device))
#         print(decoder_qo.shape)
#         print(encoder_qo.shape)
#         print("Qo Mask:",qo_mask.shape)
        
        output_sequences_mask_ex = H_QC_mask
#         print("os Mask:",output_sequences_mask_ex.shape)
        semantic_vec, _, _ = self.semantic_matching(encoder_qo.to(local_device), decoder_qo.to(local_device),
                                                    qo_mask.to(local_device), output_sequences_mask_ex.to(local_device))
        opt_score = self.option_linear(semantic_vec.to(local_device)).view(-1, self.choice_num)
        if answers is not None:
            loss = kl_ce_loss(logits, opt_score, answers, self.alpha, self.beta, t=1.0)
            return loss
        return opt_score
    

In [13]:
def eval(model, test_examples, tokenizer, eval_batch_size, choice_num, max_len, max_len_gen, external_sent_num):
    count, count_right = 0, 0
    model.eval()
    step_count = len(test_examples) // eval_batch_size
    if step_count * eval_batch_size < len(test_examples):
        step_count += 1
    step_trange = tqdm(range(step_count), desc='Eval')
    sources, targets = [], []
    for step in step_trange:
        beg_index = step * eval_batch_size
        end_index = min((step + 1) * eval_batch_size, len(test_examples))
        batch_example = [example for example in test_examples[beg_index:end_index]]
        q_ids, q_mask, qo_ids, qo_mask, clue_ids, answers, output_clue, logits, H_QC, H_QC_mask = get_input_feature(batch_example,
                                                                                           max_len, max_len_gen,
                                                                                           choice_num,
                                                                                           external_sent_num)
    
        scores = model(q_ids, q_mask, qo_ids, qo_mask, logits, H_QC, H_QC_mask)

        scores = scores.cpu().detach().tolist()
        answers = answers.cpu().detach().tolist()
        for p, a, example in zip(scores, answers, batch_example):
            p_ans = p.index(max(p))
            if p_ans == a:
                count_right += 1
            count += 1

    return count_right / count

In [14]:
choice_num = 5
lr = 1e-5
train_batch_size = 1
eval_batch_size = 1
epoch_num = 10
alpha = 0.5
beta = 0.5
max_len =  256
max_len_gen = 64 
external_sent_num = None
seed = 1
gradient_accumulation_steps = 1
num_hidden_layers = 1


In [15]:
model = StudentMC(num_hidden_layers, alpha, beta)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/modeling_t5.py:1589: FutureWarning: `T5ForConditionalGeneration.parallelize` is deprecated and will be removed in v5 of Transformers, you should load your model with `device_map='balanced'` in the call to `from_pretrained`. You can also provide your own `device_map` but it needs to be a dictionary module_name to device, so for instance {'encoder.block.0': 0, 'encoder.block.1': 1, ...}
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/t5/modeling_t5.py:930: FutureWarning: `T5Stack.parallelize` is deprecated and will be removed in v5 of Transformers, you should load your model with `device_map='balanced'` in the call to `from_pretrained`. You can also provide your own `device_map` but it needs to be a dictionary module_name to device, so for instance {'block.0': 0, 'block.1': 1, ...}
  warnings.warn(


In [16]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
output_model_path = "/kaggle/working/"
device = torch.device("cuda:0")
step_count, step_all, early_stop = 0, 0, 0
tr_loss, nb_tr_steps = 0, 0

best_dev_acc = eval(model, val_dataset, tokenizer, eval_batch_size, choice_num, max_len,
                          max_len_gen, external_sent_num)
print('best_dev_acc:', best_dev_acc)
# best_dev_acc = 0
best_test_acc = 0
for epoch in tqdm(range(epoch_num), desc='Total'):
    early_stop += 1
    order = list(range(len(train_dataset)))
    random.seed(seed + epoch)
#     random.shuffle(order)
    model.train()
    step_count = len(train_dataset) // train_batch_size
    if step_count * train_batch_size < len(train_dataset):
        step_count += 1
    step_trange = tqdm(range(step_count), desc= 'Epoch '+str(epoch+1))
    for step in step_trange:
        step_all += 1
        beg_index = step * train_batch_size
        end_index = min((step + 1) * train_batch_size, len(train_dataset))
        order_index = order[beg_index:end_index]
        batch_example = [train_dataset[index] for index in order_index]
        q_ids, q_mask, qo_ids, qo_mask, clue_ids, answers, output_clue, logits, H_QC, H_QC_mask = get_input_feature(batch_example,
                                                                                           max_len, max_len_gen,
                                                                                           choice_num,
                                                                                           external_sent_num)
    
        loss = model(q_ids, q_mask, qo_ids, qo_mask, logits, H_QC, H_QC_mask, answers = answers)

        loss = loss.mean()
        tr_loss += loss.item()
        nb_tr_steps += 1
        loss = loss / gradient_accumulation_steps
        loss.backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            # scheduler.step()
            optimizer.zero_grad()

        loss_show = ' Epoch:' + str(epoch) + " loss:" + str(round(tr_loss / nb_tr_steps, 4))
        step_trange.set_postfix_str(loss_show)

    dev_acc = eval(model, val_dataset, tokenizer, eval_batch_size,
                                                 choice_num, max_len, max_len_gen,
                                                 external_sent_num)
    print('dev_acc:', dev_acc)
    if dev_acc > best_dev_acc:
        early_stop = 0
        test_acc = eval(model, test_dataset, tokenizer, eval_batch_size,
                                                        choice_num, max_len, max_len_gen,
                                                        external_sent_num)
        best_dev_acc, best_test_acc = dev_acc, test_acc

        save_model(output_model_path, model, optimizer)
        print('new best dev acc:', dev_acc, 'test_acc:', test_acc)

    if early_stop >= 10:
        break

print('best dev acc:', best_dev_acc, 'best_test_acc:', best_test_acc)

Eval:   0%|          | 0/500 [00:00<?, ?it/s]

best_dev_acc: 0.25


Total:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/2478 [00:00<?, ?it/s]

Eval:   0%|          | 0/500 [00:00<?, ?it/s]

...

Eval:   0%|          | 0/500 [00:00<?, ?it/s]

dev_acc: 0.428


Epoch 10:   0%|          | 0/2478 [00:00<?, ?it/s]

Eval:   0%|          | 0/500 [00:00<?, ?it/s]

dev_acc: 0.432
best dev acc: 0.452 best_test_acc: 0.43
